In [69]:
import setup # resolve path to 'src'
import numpy as np
import pandas as pd

from typing import Optional
from build_parse import *
from metrics import *

pd.set_option('display.max_rows', None)

In [70]:
progs = [ CoreutilsProgram(progname) for progname in COREUTILS_PROG_NAMES ]
prognames = [ prog.get_name() for prog in progs ]

prognames_analyze = [ "stat", "nohup", "pinky", "csplit", "ginstall", "fmt", "df", "join", "expr", "seq", "unexpand", "tsort", "tee", "base64", "sum", "cksum", "wc" ]
progs_analyze = []
for progname in prognames_analyze:
    for prog in progs:
        if progname == prog.get_name():
            progs_analyze.append(prog)
            break

def prog_from_progname(progname: str) -> Program:
    for prog in progs:
        if progname == prog.get_name():
            return prog

# Define the build options to test for each program
debug_opts = BuildOptions(debug=True, strip=False, optimization=0)
standard_opts = BuildOptions(debug=False, strip=False, optimization=0)
strip_opts = BuildOptions(debug=False, strip=True, optimization=0)

opts_sets = (debug_opts, standard_opts, strip_opts)

# Get the parser functions
dwarf_parser = get_parser("dwarf")
ghidra_parser = get_parser("ghidra")

# ensure that each program is built according to all variations of build options
for prog in progs:
    for opts in (debug_opts, standard_opts, strip_opts):
        assert(prog.valid_build(opts))

In [71]:

# the filename format for saving parsed ProgramInfo pickle objects
def mangle_proginfo_save_name(parsername: str, prog: Program, opts: BuildOptions) -> str:
    return "{}.{}.pickle".format(prog.get_binary_name(opts), parsername)

def get_proginfo_save_path(parsername: str, prog: Program, opts: BuildOptions) -> Path:
    return PICKLE_CACHE_DIR.joinpath(mangle_proginfo_save_name(parsername, prog, opts))

def save_proginfo(proginfo: ProgramInfo, parsername: str, prog: Program, opts: BuildOptions):
    save_pickle(proginfo, get_proginfo_save_path(parsername, prog, opts))

def load_proginfo(parsername: str, prog: Program, opts: BuildOptions) -> ProgramInfo:
    return load_pickle(get_proginfo_save_path(parsername, prog, opts))

# the filename format for saving UnoptimizedProgramInfoCompare2 objects
def mangle_cmp_save_name(prog: Program, opts: BuildOptions) -> str:
    return "{}.cmp.pickle".format(prog.get_binary_name(opts))

def get_cmp_save_path(prog: Program, opts: BuildOptions) -> Path:
    return PICKLE_CACHE_DIR.joinpath(mangle_cmp_save_name(prog, opts))

def save_cmp(cmp: UnoptimizedProgramInfoCompare2, prog: Program, opts: BuildOptions):
    save_pickle(cmp, get_cmp_save_path(prog, opts))

def load_cmp(prog: Program, opts: BuildOptions) -> UnoptimizedProgramInfoCompare2:
    return load_pickle(get_cmp_save_path(prog, opts))

In [72]:
# DWARF: only parse with the debug build options
# Ghidra: parse with all variations of build options
# Cache the results in local pickle_cache directory, named based on the 'mangle' scheme

reparse = False # should we re-parse even if we already parsed and cached a program?
skip_parsing = True # should we skip the parsing? set to True if we already parsed & cached

class ParseException(Exception):
    pass

def parse(parser: Callable, prog: Program, opts: BuildOptions) -> Optional[ProgramInfo]:
    try:
        return parser(prog.get_binary_path(opts))
    except:
        return None

failed = []
if not skip_parsing:
    for prog in progs:
        dwarf_debug_savepath = get_proginfo_save_path("dwarf", prog, debug_opts)
        if reparse or not dwarf_debug_savepath.exists():
            dwarf_debug = parse(dwarf_parser, prog, debug_opts)
            if dwarf_debug is None:
                failed.append(("dwarf", prog.get_name(), debug_opts))
            else:
                save_pickle(dwarf_debug, dwarf_debug_savepath)

        for opts in opts_sets:
            ghidra_parse_savepath = get_proginfo_save_path("ghidra", prog, opts)
            if reparse or not dwarf_debug_savepath.exists():
                ghidra_parse = parse(ghidra_parser, prog, opts)
                if ghidra_parse is None:
                    failed.append(("ghidra", prog.get_name(), opts))
                else:
                    save_pickle(ghidra_parse, ghidra_parse_savepath)

In [73]:
print(failed)

for prog in progs:
    for opts in opts_sets:
        assert(get_proginfo_save_path("ghidra", prog, opts).exists())

[]


In [74]:
# For each program & build options combination, compute & store comparison object

recompare = False
skip_comparisons = True

failed = []
if not skip_comparisons:
    for prog in progs:
        # load the DWARF ground-truth ProgramInfo
        dwarf_proginfo = load_pickle(get_proginfo_save_path("dwarf", prog, debug_opts))
        assert(dwarf_proginfo is not None)

        # for each set of compilation options, load the Ghidra decompiler ProgramInfo
        # then compute & store the comparison object
        for opts in (strip_opts,):
            cmp_save_path = get_cmp_save_path(prog, opts)
            if recompare or not cmp_save_path.exists():
                ghidra_proginfo = load_pickle(get_proginfo_save_path("ghidra", prog, opts))
                assert(ghidra_proginfo is not None)
                try:
                    cmp = compare2(dwarf_proginfo, ghidra_proginfo)
                    save_pickle(cmp, get_cmp_save_path(prog, opts))
                except:
                    failed.append((prog.get_name(), opts))

In [75]:
print(failed)

for prog in progs:
    for opts in opts_sets:
        assert(get_cmp_save_path(prog, opts).exists())

[]


In [76]:
# For each opts, compute the tables

def mangle_table_save_name(
    tablename: str,
    opts: BuildOptions
) -> str:
    return "{}{}.csv".format(tablename, suffix(opts))

def mangle_table_display_name(
    tablename: str,
    opts: BuildOptions
) -> str:
    def _suffix(opts: BuildOptions) -> str:
        return "(optimization={}, stripped={}, debug={})".format(opts.optimization, opts.strip, opts.debug)

    return "{} {}".format(tablename, _suffix(opts))

def get_table_save_path(
    tablename: str,
    opts: BuildOptions
) -> Path:
    return DATA_DIR.joinpath(mangle_table_save_name(tablename, opts))

def load_table(
    tablename: str,
    opts: BuildOptions
) -> pd.DataFrame:
    return pd.read_csv(get_table_save_path(tablename, opts), index_col=0)

def load_table_filter_analyzed(tablename: str, opts: BuildOptions) -> pd.DataFrame:
    return load_table(tablename, opts).filter(prognames_analyze, axis=0)

In [77]:
recompute = False
skip_compute_metrics = True

metrics_groups = make_metrics()

if not skip_compute_metrics:
    for opts in opts_sets:
        cmps = [ load_cmp(prog, opts) for prog in progs ]
        for grp in metrics_groups:
            save_path = get_table_save_path(grp.get_name(), opts)
            tablename = mangle_table_display_name(grp.get_display_name(), opts)
            print(tablename)
            if recompute or not save_path.exists():
                df = compute_comparisons_metrics_dataframe(prognames, cmps, grp.get_metrics())
                df.to_csv(save_path)


In [78]:
bytes_group = metrics_groups[0]
functions_group = metrics_groups[1]
varnodes_group = metrics_groups[2]
decomposed_varnodes_group = metrics_groups[9]
array_comparisons_group = metrics_groups[13]

primitive_metatypes = [MetaType.INT, MetaType.FLOAT, MetaType.POINTER]
complex_metatypes = [MetaType.ARRAY, MetaType.STRUCT, MetaType.UNION]

def varnodes_group_metatype(metatype: int) -> MetricsGroup:
    _map = dict([ (meta, i) for i, meta in enumerate(primitive_metatypes + complex_metatypes, 3) ])
    return metrics_groups[_map[metatype]]

varnodes_groups_metatypes = [ varnodes_group_metatype(metatype) for metatype in (primitive_metatypes + complex_metatypes) ]

def decomposed_varnodes_group_metatype(metatype: int) -> MetricsGroup:
    _map = dict([ (meta, i) for i, meta in enumerate(primitive_metatypes, 10) ])
    return metrics_groups[_map[metatype]]

decomposed_varnodes_groups_metatypes = [ decomposed_varnodes_group_metatype(metatype) for metatype in primitive_metatypes ]

high_varnodes_groups = [varnodes_group] + varnodes_groups_metatypes
decomposed_varnodes_groups = [decomposed_varnodes_group] + decomposed_varnodes_groups_metatypes

In [79]:
def get_table(
    grp: MetricsGroup,
    opts: BuildOptions,
    analyzed_only: bool = True
)-> pd.DataFrame:
    df = load_table(grp.get_name(), opts)
    return df if not analyzed_only else df.filter(prognames_analyze, axis=0)

def display_analyzed_tables(
    metrics_groups: List[MetricsGroup],
    opts_sets: List[BuildOptions],
    analyzed_only: bool = True
):
    for grp in metrics_groups:
        for opts in opts_sets:
            table_display_name = mangle_table_display_name(grp.get_display_name(), opts)
            df = get_table(grp, opts, analyzed_only=analyzed_only)

            print("{} {} {}".format("-"*10, table_display_name, "-"*10))
            display(df)

In [87]:
for grp in (varnodes_group,):
    for opts in (strip_opts,):
        df = get_table(grp, opts)
        df["Varnodes fraction partially recovered"] = df.iloc[:,2:6].sum(axis=1) / df.iloc[:,0]
        df["Varnodes fraction exactly recovered"] = df.iloc[:,5] / df.iloc[:,0]
        display(df)

        tmp = df.iloc[:,6:].mean(axis=0)
        display(tmp)

,Ground truth varnodes,Varnodes matched @ level=NO_MATCH,Varnodes matched @ level=OVERLAP,Varnodes matched @ level=SUBSET,Varnodes matched @ level=ALIGNED,Varnodes matched @ level=MATCH,"Varnode average comparison score [0,1]",Varnodes fraction partially recovered,Varnodes fraction exactly recovered
stat,608,5,21,14,321,247,0.822368,0.991776,0.406250
nohup,162,1,7,4,105,45,0.787037,0.993827,0.277778
pinky,182,1,11,2,110,58,0.792582,0.994505,0.318681
csplit,1003,8,41,15,544,395,0.818295,0.992024,0.393819
fmt,186,1,6,2,110,67,0.817204,0.994624,0.360215
df,646,1,20,11,287,327,0.855650,0.998452,0.506192
join,260,1,8,3,151,97,0.822115,0.996154,0.373077
expr,932,8,37,25,499,363,0.814378,0.991416,0.389485
seq,279,1,20,6,128,124,0.817204,0.996416,0.444444
unexpand,158,1,6,2,92,57,0.813291,0.993671,0.360759


Varnode average comparison score [0,1]    0.808120
Varnodes fraction partially recovered     0.985349
Varnodes fraction exactly recovered       0.376531
dtype: float64

In [102]:
varnode_compare_levels = list(VarnodeCompareLevel.range())
varnode_compare_level_labels = [ VarnodeCompareLevel.to_string(level) for level in varnode_compare_levels ]

metatypes = primitive_metatypes + complex_metatypes
metatype_labels = [ MetaType.repr(metatype) for metatype in metatypes ]

seriess = []
for metatype in metatypes:
    metatype_str = MetaType.repr(metatype)
    grp = varnodes_group_metatype(metatype)
    df = get_table(grp, strip_opts)
    metatype_varnodes = df.iloc[:,0].sum()
    varnodes_by_levels = df.iloc[:,1:6].sum(axis=0)
    varnodes_by_levels.index = varnode_compare_level_labels
    level_ratios = varnodes_by_levels / metatype_varnodes
    seriess.append(level_ratios)
    # print(metatype_str)
    # print(metatype_varnodes)
    # display(varnodes_by_levels)
    # display(level_ratios)

df = pd.DataFrame(seriess, index=metatype_labels, columns=varnode_compare_level_labels)
df

,NO_MATCH,OVERLAP,SUBSET,ALIGNED,MATCH
INT,0.007951,0.000795,0.000000,0.571959,0.419295
FLOAT,0.000000,0.485714,0.000000,0.371429,0.142857
POINTER,0.016275,0.000603,0.000000,0.608800,0.374322
ARRAY,0.222760,0.203390,0.152542,0.019370,0.401937
STRUCT,0.000000,0.408284,0.144970,0.349112,0.097633
UNION,NaN,NaN,NaN,NaN,NaN


In [81]:
analyzed_opts_sets = (strip_opts, debug_opts)

display_analyzed_tables(
    (varnodes_group,),
    analyzed_opts_sets,
    analyzed_only=True
)

---------- VARNODES (optimization=0, stripped=True, debug=False) ----------


,Ground truth varnodes,Varnodes matched @ level=NO_MATCH,Varnodes matched @ level=OVERLAP,Varnodes matched @ level=SUBSET,Varnodes matched @ level=ALIGNED,Varnodes matched @ level=MATCH,"Varnode average comparison score [0,1]"
stat,608,5,21,14,321,247,0.822368
nohup,162,1,7,4,105,45,0.787037
pinky,182,1,11,2,110,58,0.792582
csplit,1003,8,41,15,544,395,0.818295
fmt,186,1,6,2,110,67,0.817204
df,646,1,20,11,287,327,0.855650
join,260,1,8,3,151,97,0.822115
expr,932,8,37,25,499,363,0.814378
seq,279,1,20,6,128,124,0.817204
unexpand,158,1,6,2,92,57,0.813291


---------- VARNODES (optimization=0, stripped=False, debug=True) ----------


,Ground truth varnodes,Varnodes matched @ level=NO_MATCH,Varnodes matched @ level=OVERLAP,Varnodes matched @ level=SUBSET,Varnodes matched @ level=ALIGNED,Varnodes matched @ level=MATCH,"Varnode average comparison score [0,1]"
stat,608,0,0,1,0,607,0.999178
nohup,162,0,0,0,0,162,1.000000
pinky,182,0,0,1,0,181,0.997253
csplit,1003,0,1,1,6,995,0.997258
fmt,186,0,0,0,0,186,1.000000
df,646,0,2,0,0,644,0.997678
join,260,0,0,0,0,260,1.000000
expr,932,0,1,1,6,924,0.997049
seq,279,0,0,0,0,279,1.000000
unexpand,158,0,0,0,0,158,1.000000


In [82]:
cmp = load_cmp(prog_from_progname("cksum"), debug_opts)
truth = sum([ varnode.get_size() for varnode in varnodes_truth(cmp) ])
missed = sum([ varnode.get_size() for varnode in varnodes_missed(cmp) ])
overlapped = varnode_compare_records_matched_at_level(cmp, VarnodeCompareLevel.OVERLAP)
for varnode in varnodes_missed(cmp):
    print(varnode.get_var().get_parent_function().get_name())

# for record in overlapped:
#     print(record.get_varnode().get_var().get_parent_function().get_name())
# print(overlapped)

cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul
cksum_pclmul